In [1]:
# installing catboost
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# imports
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
import catboost as cb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [3]:
# loading the dataset
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [4]:
# getting the info of dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [5]:
# droping the cat. features with nulls for catboost 
df.drop(['embarked', 'embark_town', 'deck'], axis=1, inplace=True)

In [6]:
# checking missing values, nulls in num features will be handled by catboost
df.isnull().sum()

survived        0
pclass          0
sex             0
age           177
sibsp           0
parch           0
fare            0
class           0
who             0
adult_male      0
alive           0
alone           0
dtype: int64

In [7]:
# creating X & y
X = df.drop(['survived'], axis=1)
y = df['survived']

In [8]:
# splitting the dataset into train & test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

In [9]:
# getting the shape of the splits
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(712, 11) (179, 11) (712,) (179,)


In [10]:
# selecting the cat features
cat = df.select_dtypes(['object', 'category']).columns.to_list()

In [11]:
# training a model
model = cb.CatBoostClassifier(n_estimators=500, learning_rate=.03, loss_function='Logloss', random_state=20, 
                              nan_mode='Min', bootstrap_type='Bayesian', task_type='GPU')
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], cat_features=cat, verbose=100, early_stopping_rounds=50)

0:	learn: 0.6079279	test: 0.6076840	best: 0.6076840 (0)	total: 44.6ms	remaining: 22.2s
100:	learn: 0.0009648	test: 0.0009331	best: 0.0009331 (100)	total: 2s	remaining: 7.89s
200:	learn: 0.0002782	test: 0.0002668	best: 0.0002668 (200)	total: 3.67s	remaining: 5.46s
300:	learn: 0.0001535	test: 0.0001467	best: 0.0001467 (300)	total: 5.33s	remaining: 3.52s
400:	learn: 0.0001036	test: 0.0000989	best: 0.0000989 (400)	total: 8.2s	remaining: 2.02s
499:	learn: 0.0000780	test: 0.0000745	best: 0.0000745 (499)	total: 14.4s	remaining: 0us
bestTest = 7.446602445e-05
bestIteration = 499


In [12]:
# checking the accuracy
print('Accuracy:', (accuracy_score(y_test, model.predict(X_test)) * 100).round(2))

Accuracy: 100.0


In [13]:
# confusion matrix
print(confusion_matrix(y_test, model.predict(X_test)))

[[112   0]
 [  0  67]]


In [14]:
# printing a clf report
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       112
           1       1.00      1.00      1.00        67

    accuracy                           1.00       179
   macro avg       1.00      1.00      1.00       179
weighted avg       1.00      1.00      1.00       179

